In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os
from PIL import Image

from tqdm import tqdm
from cropping_functions import *

In [3]:
from PIL.ImageOps import flip, mirror
import PIL

In [4]:
# config variables
valid_file_types = ['.jpg'] # ignore other files
folder_to_scan = './hirise_scrapped/'

cropping_window = 512
saving_folder_name = './cropped_files/'
padding_tresh = 0.25
resize = False
image_output_size = 512
remove_top_border = True

os.makedirs(saving_folder_name, exist_ok=True)

In [6]:
valid_files = [os.path.join(folder_to_scan, f) for f in os.listdir(folder_to_scan) if os.path.splitext(f)[1] in valid_file_types]
print(len(valid_files))

16


In [7]:
def remove_top_border_from_image(file_path):
    """
    use this function to remove top red border from hirise images.
    """
    file_name = os.path.basename(file_path)
    base_name, file_ext = os.path.splitext(file_name)
    im = Image.open(file_path)

    im_w = im.size[0]
    im_h = im.size[1]
    # delete superior border
    imc = im.crop((0,40,im_w,im_h)) #left bottom right upper

#     f_name = saving_folder_name + base_name + file_ext
#     imc.save(f_name)
    return imc

In [8]:
def image_square_resize(im_input, new_size):
    im = im_input.copy()
    im = im.resize((new_size, new_size), PIL.Image.ANTIALIAS)    
    
    return im


def image_shifter(im_input, left_shift, bottom_shift):
        shift_box = (left_shift, bottom_shift ,im_input.size[0], im_input.size[1])
        return im_input.crop(shift_box)

    
def image_rotater(im_input, angle):
    if angle==90:
        return im_input.transpose(Image.ROTATE_90) 
    elif angle == 180:
        return im_input.transpose(Image.ROTATE_180) 
    elif angle == 270:
        return im_input.transpose(Image.ROTATE_270) 
    elif angle == 0:
        return im_input
    else:
        raise ValueError('No agle supported')
        
        
def image_augmentator(im_input, return_orig = True):
    im_aug = []
    if return_orig:
        im_aug.append(im_input.copy())
    # 1.FLIP
    im_aug.append(flip(im_input.copy()))
    # 2. rot 180
    im_aug.append(image_rotater(im_input.copy(), 180))
    # 3. Flip(rot_90)
    im_aug.append(flip(image_rotater(im_input.copy(), 90)))
    # 4. (flip(rot_270))
    im_aug.append(flip(image_rotater(im_input.copy(), 270)))
    return im_aug     


def random_augmentator(im_input, return_orig = True):
    im_aug = []
    transfm = []
    if return_orig:
        im_aug.append(im_input.copy())
        transfm.append('None')
    # 1.FLIP
    for i in range(4):
        augmented = im_input.copy()
        t = ''
        if np.random.choice([True, False]):
            augmented = flip(augmented)
            t += '_flip'
        if np.random.choice([True, False]):
            augmented = mirror(augmented)
            t += '_mirror'
        angle = np.random.choice([90, 180, 270])
        augmented = image_rotater(augmented, angle)
        t += '_{}'.format(angle)
        im_aug.append(augmented)
        transfm.append(t)
    return im_aug, transfm

### No augmentation

In [9]:
# for file in tqdm(valid_files):
#     # load file
#     if remove_top_border:
#         im = remove_top_border_from_image(file)
#     else:
#         im = Image.open(file)
    
#     # generate cropping boxes for file
#     croping_boxes = generate_cropping_boxes(im, cropping_window, padding_tresh) 
#     file_name = os.path.basename(file)
#     base_name, file_ext = os.path.splitext(file_name)
#     # crop
#     for i,b in enumerate(croping_boxes):
#         imc = im.crop(b) #left bottom right upper
#         if resize: 
#             imc = image_square_resize(imc,image_output_size )
#         f_name = saving_folder_name + base_name +'_{}_'.format(i) + file_ext
#         imc.save(f_name)   

### Aumentation

In [10]:
for file in tqdm(valid_files):
    # load file
    if remove_top_border:
        im = remove_top_border_from_image(file)
    else:
        im = Image.open(file)
    for ia, im in enumerate(image_augmentator(im)): # note that we iterate here over image_augmentator(im).
        # generate cropping boxes for file
        croping_boxes = generate_cropping_boxes(im, cropping_window, padding_tresh) 

        file_name = os.path.basename(file)
        base_name, file_ext = os.path.splitext(file_name)
        for i,b in enumerate(croping_boxes):
            imc = im.crop(b) #left bottom right upper
            if resize: 
                imc = image_square_resize(imc,image_output_size )
            f_name = saving_folder_name + base_name +'_{}_{}_'.format(ia,i) + file_ext
            imc.save(f_name)

100%|██████████| 16/16 [00:18<00:00,  1.13s/it]
